In [13]:
import re
import json
from genericpath import exists
import os
import pandas as pd

In [2]:
nom="pit"
path_table=os.walk(f"/workspaces/F1_website/data/raw/{nom}")

df=pd.DataFrame()
for fichiers in path_table:
    for fichier in fichiers[2]:
        with open(fichiers[0]+"/"+fichier) as f:
            data = json.load(f)
            df=pd.concat([df,pd.json_normalize(data)])

display(df)

/tmp/ipykernel_5176/3175578739.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df=pd.concat([df,pd.json_normalize(data)])


,date,session_key,driver_number,meeting_key,lap_number,pit_duration,lane_duration,stop_duration
0,2024-02-21T07:00:04.303000+00:00,9462,63,1228,1,NaN,NaN,NaN
1,2024-02-21T07:00:19.465000+00:00,9462,14,1228,1,NaN,NaN,NaN
2,2024-02-21T07:00:24.249000+00:00,9462,23,1228,1,NaN,NaN,NaN
3,2024-02-21T07:00:30.814000+00:00,9462,77,1228,1,NaN,NaN,NaN
4,2024-02-21T07:00:41.497000+00:00,9462,31,1228,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9597,2025-07-06T15:26:39.555000+00:00,9947,16,1277,42,28.840,28.840,2.6
9598,2025-07-06T15:26:43.790000+00:00,9947,23,1277,42,29.200,29.200,NaN
9599,2025-07-06T15:26:45.946000+00:00,9947,31,1277,42,28.646,28.646,2.5
9600,2025-07-06T15:27:27.684000+00:00,9947,81,1277,43,40.280,40.280,14.6


In [15]:
display(df.loc[~df["pit_duration"].isna()])

,date,session_key,driver_number,meeting_key,lap_number,pit_duration,lane_duration,stop_duration
9,2024-02-21T07:03:30.378000+00:00,9462,31,1228,2,30.400,30.400,NaN
11,2024-02-21T07:10:28.057000+00:00,9462,77,1228,2,471.200,471.200,NaN
12,2024-02-21T07:10:39.250000+00:00,9462,81,1228,2,29.300,29.300,NaN
13,2024-02-21T07:11:02.263000+00:00,9462,63,1228,7,34.600,34.600,NaN
14,2024-02-21T07:11:21.229000+00:00,9462,23,1228,3,424.200,424.200,NaN
...,...,...,...,...,...,...,...,...
9597,2025-07-06T15:26:39.555000+00:00,9947,16,1277,42,28.840,28.840,2.6
9598,2025-07-06T15:26:43.790000+00:00,9947,23,1277,42,29.200,29.200,NaN
9599,2025-07-06T15:26:45.946000+00:00,9947,31,1277,42,28.646,28.646,2.5
9600,2025-07-06T15:27:27.684000+00:00,9947,81,1277,43,40.280,40.280,14.6


In [31]:
nom="race_control"
with open(f"/workspaces/F1_website/data/raw/{nom}/{nom}.json") as f:
    df=pd.json_normalize(json.load(f))

df2=df.loc[((df["category"].isin(["Flag","SafetyCar"]))&(~df["flag"].isin(["BLUE","BLACK AND WHITE"])))|((df["category"]=="Other") & (df["message"].str.contains("SECOND TIME PENALTY")) & ~df["message"].str.contains("PENALTY SERVED")),["session_key","lap_number","category","flag","scope","sector","qualifying_phase","message"]].drop_duplicates()

df2["driver_number"] = df2["message"].str.extract(r'PENALTY FOR CAR ([0-9]*) ').astype('Int64') # il faut créer pour le numéro du pilote lors des penalites
df2["lap_number"] = df2["lap_number"].astype('Int64')
df2["sector"] = df2["sector"].astype('Int64')
df2["qualifying_phase"] = df2["qualifying_phase"].astype('Int64')

display(df2) #completer les numero de pilots

,session_key,lap_number,category,flag,scope,sector,qualifying_phase,message,driver_number
2,7763,<NA>,Flag,GREEN,Track,<NA>,<NA>,GREEN LIGHT - PIT EXIT OPEN,<NA>
4,7763,<NA>,Flag,DOUBLE YELLOW,Sector,6,<NA>,DOUBLE YELLOW IN TRACK SECTOR 6,<NA>
5,7763,<NA>,Flag,YELLOW,Sector,5,<NA>,YELLOW IN TRACK SECTOR 5,<NA>
6,7763,<NA>,SafetyCar,None,None,<NA>,<NA>,VIRTUAL SAFETY CAR DEPLOYED,<NA>
7,7763,<NA>,Flag,RED,Track,<NA>,<NA>,RED FLAG,<NA>
...,...,...,...,...,...,...,...,...,...
17305,9947,44,Flag,CLEAR,Sector,7,<NA>,CLEAR IN TRACK SECTOR 7,<NA>
17311,9947,44,Flag,YELLOW,Sector,15,<NA>,YELLOW IN TRACK SECTOR 15,<NA>
17314,9947,44,Flag,CLEAR,Sector,15,<NA>,CLEAR IN TRACK SECTOR 15,<NA>
17318,9947,46,Flag,CLEAR,Sector,7,<NA>,CLEAR IN TRACK SECTOR 7,<NA>


In [47]:
table_nation=df[["full_name","country_code"]][~df["country_code"].isnull()].drop_duplicates()

table=df[["meeting_key","driver_number", "full_name","name_acronym","team_name","headshot_url"]].drop_duplicates()

table = pd.merge(table, table_nation, on=["full_name"], how="left")

display(table[table["full_name"]=="Max VERSTAPPEN"].drop_duplicates())

,meeting_key,driver_number,full_name,name_acronym,team_name,headshot_url,country_code
0,1140,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
20,1141,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
40,1142,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
60,1143,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
80,1207,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
...,...,...,...,...,...,...,...
1486,1273,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1507,1274,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1528,1275,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1549,1276,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED


In [ ]:
display(df[["full_name","country_code"]][df["full_name"]=="Max VERSTAPPEN"].drop_duplicates())

,full_name,country_code
0,Max VERSTAPPEN,NED
4723,Max VERSTAPPEN,None


In [5]:
display(df[["meeting_key","team_name","team_colour"]].drop_duplicates())

,meeting_key,team_name,team_colour
0,1140,Red Bull Racing,3671C6
1,1140,Williams,37BEDD
2,1140,McLaren,F58020
3,1140,Alpine,2293D1
5,1140,Aston Martin,358C75
...,...,...,...
7083,1277,Mercedes,00D7B6
7084,1277,Aston Martin,229971
7085,1277,Ferrari,ED1131
7087,1277,Williams,1868DB
